In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="MIG-195f63f2-e855-5f08-80cd-5689c4e14928" 
from hydra.utils import instantiate
import yaml
import torch
import numpy as np
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
device = "cuda:0"

In [ ]:
from src.data.get_datamodules import get_data
from cyto_dl.models.utils.mlflow import load_model_from_checkpoint
from src.models.save_embeddings import get_pc_loss

def get_data_and_models(dataset_name, batch_size):
    data = get_data(dataset_name, batch_size)
    # data_list = [data[0], data[0], data[1], data[-2], data[-2]]
    data_list = [data[-1]]

    # run_ids = [
    #     "6bb8003c781e4661914a768763a352a3",
    #     "68157cde6680432292c3f5b959394bec",
    #     "e41f0bbf26944ba9b9e4541e87bf42f0",
    #     "358cef46219c435ab1b16d2a06dd4436",
    # ]
    run_ids = [
        "f6fbe34d298c41af8b07b7ddd30aaea8",
    ]
    run_names = ["vit"]
    all_models = []
    for i in run_ids:
        all_models.append(
            load_model_from_checkpoint(
                "https://mlflow.a100.int.allencell.org",
                i,
                path="checkpoints/val/loss/best.ckpt",
                strict=False,
            )
        )
    return data_list, all_models, run_names

In [ ]:
dataset_name = 'pcna'
batch_size = 1
data_list, all_models, run_names = get_data_and_models(dataset_name, batch_size)

In [ ]:
run_names

In [ ]:
all_models.reverse()
data_list.reverse()
run_names.reverse()

In [ ]:
run_names

In [ ]:
from src.models.save_embeddings import save_embeddings

debug = False
path = "./embeddings_pcna_vit"
outs = save_embeddings(path, data_list, all_models, run_names, debug, device=device)

In [ ]:
from src.models.save_embeddings import save_emissions

debug = False
path = "./emissions_pcna_vit"
save_emissions(path, data_list, all_models, run_names, 20, debug, device)

In [ ]:
import pandas as pd
emissions = pd.read_csv(path + '/emissions.csv')
import seaborn as sns
g = sns.catplot(data=emissions,x='model', y='emissions', kind='point')
g.set_xticklabels(rotation=30)


In [ ]:
from src.models.compute_features import compute_features

save_folder = './features_pcna_vit/'
# keys = ['image', 'image', 'pcloud', 'pcloud']
keys = ['image']
image_config_list = [
        f"../data/configs/inference_pcna_data_configs/mae.yaml",
    ]

modality_list = ['image']
splits_list = ['train', 'val', 'test']
# split_list_evolve = ['test']
compute_features(
    dataset='pcna_vit',
    save_folder=save_folder,
    data_list=data_list,
    all_models=all_models,
    run_names=run_names,
    keys=keys,
    device="cuda:0",
    max_embed_dim=256,
    class_label='cell_stage_fine',
    num_evolve_samples=40,
    compute_embeds=True,
    splits_list=splits_list,
    config_list_evolve=image_config_list,
    modality_list_evolve=modality_list,)